In [ ]:
import pandas as pd
import re

In [ ]:
tcof = pd.read_csv(r"F:\Corpus\finaux\tcof_NEW.tsv", sep='\t')

In [ ]:
tcof

In [ ]:
tcof['index_corpus']=[x.lower() for x in tcof['corpus'].tolist()]

In [ ]:
tcof['index_enregistrement']= [x.lower() +'_'+ y.lower() for x,y in list(zip(tcof['corpus'].tolist(), tcof['sous_corpus'].tolist()))]

In [ ]:
tcof['index_locuteur']=[x.lower() +'_'+y.lower() for x,y in list(zip(tcof['index_enregistrement'].tolist(), tcof['speaker'].tolist())) ]

In [ ]:
tcof['nb_index_enregistrement'] = tcof['index_enregistrement'].ne(tcof['index_enregistrement'].shift()).cumsum()

In [ ]:
tcof['numero_tour'] = tcof.groupby('nb_index_enregistrement').cumcount() + 1

In [ ]:
tcof['index_tour'] = [x.lower() +'_'+ str(y) for x,y in list(zip(tcof['index_enregistrement'].tolist(), tcof['numero_tour'].tolist()))]

In [ ]:
tcof

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
content_french = ["Les astronomes amateurs jouent également un rôle important en recherche; les plus sérieux participant couramment au suivi d'étoiles variables, à la découverte de nouveaux astéroïdes et de nouvelles comètes, etc.", 'Séquence vidéo.', "John Richard Bond explique le rôle de l'astronomie."]
for i in content_french:
        print(i)
        print(list(enumerate((word_tokenize(i, language='french')))))

In [ ]:
def tokenize(x):
    if isinstance(x, str):
        return list(enumerate(word_tokenize(x, language='french')))
    else:
        return []

In [ ]:
tcof['tokens']=tcof['text_clean'].apply(lambda x : tokenize(x))

In [ ]:
tcof

In [ ]:
def create_token_index(tokens, index_tour):
    index_tokens = [str(index_tour)+'_'+str(ind) for ind, tok in tokens]
    if len(tokens)==len(index_tokens):
        return index_tokens


In [ ]:
tcof['tokens']=tcof['text_clean'].apply(lambda x : tokenize(x))

In [ ]:
tcof['index_token'] = tcof.apply(lambda x: create_token_index(x['tokens'], x['index_tour']), axis=1)

In [ ]:
tcof['type']='oral'

In [ ]:
tcof['date']=''

In [ ]:
tcof['path_xml']=tcof['path_trs'].apply(lambda x : x.replace('.trs','.xml'))

In [ ]:
tcof

# prepare tables to inject into database

In [ ]:
tcof_corpus = tcof[['index_corpus','corpus','date','type']]

tcof_corpus = tcof_corpus.groupby('index_corpus').agg({
    'corpus': 'first',
     'date': 'first',
     'type': 'first'
    }).reset_index()


In [ ]:
tcof_corpus

In [ ]:
tcof_enregistrement = tcof[['index_enregistrement','sous_corpus','description','date_enregistrement','responsable','path_trs','path_wav','index_corpus']]#'path_xml',

tcof_enregistrement = tcof_enregistrement.groupby('index_enregistrement').agg({
    'sous_corpus': 'first',
    'description': 'first',
    'date_enregistrement': 'first',
    'responsable' : 'first',
    'path_trs' : 'first',
    'path_wav':'first',
    'index_corpus': 'first'
}).reset_index()

In [ ]:
tcof_enregistrement

In [ ]:
tcof.columns

In [ ]:
tcof_tour = tcof[['index_tour', 'text_clean','startTime','endTime','index_locuteur', 'index_enregistrement']]
tcof_tour = tcof_tour.groupby('index_tour').agg({
    'text_clean': 'first',
    'startTime': 'first',
    'endTime' : 'first',
    'index_locuteur' : 'first',
    'index_enregistrement':'first'
}).reset_index()

In [ ]:
tcof_tour

In [ ]:
tcof_locuteurs = tcof[['index_locuteur', 'speaker_name', 'age_speaker', 'etude_speaker', 'profession_speaker','index_enregistrement']]

In [ ]:
tcof_locuteurs = tcof_locuteurs.groupby('index_locuteur').agg({
    'speaker_name': 'first',
    'age_speaker': 'first',
    'etude_speaker': 'first',
    'profession_speaker': 'first',
    'index_enregistrement': 'first'
}).reset_index()

In [ ]:
tcof.columns

In [ ]:
tcof_locuteurs

In [ ]:
tcof.head()

In [ ]:
tcof_tokens = tcof[['index_token', 'tokens', 'index_tour', 'index_locuteur']]

In [ ]:
tcof_tokens = tcof_tokens.explode('index_token')

In [ ]:
tcof_tokens = tcof_tokens[tcof_tokens['index_token'].notna()]

In [ ]:
def extract_nb(tokens, index_token):
    pattern = '^.*_(\d*)$'
    try : 
        match = re.match(pattern, index_token)
    except:
        print(index_token, type(index_token))
    if match :
        index = match.group(1)
        for i, (num, forme) in enumerate(tokens):
            if i == int(index):
                return num
    else:
        print(index_token)
        
def extract_forme(tokens, index_token):
    pattern = '^.*_(\d*)$'
    try : 
        match = re.match(pattern, index_token)
    except:
        print(index_token, type(index_token))
    if match :
        index = match.group(1)
        for i, (num, forme) in enumerate(tokens):
            if i == int(index):
                return forme
    else:
        print(index_token)

In [ ]:
tcof_tokens['forme']= tcof_tokens.apply(lambda row : extract_forme(row['tokens'], row['index_token']), axis=1)
tcof_tokens['num_tokens'] = tcof_tokens.apply(lambda row : extract_nb(row['tokens'], row['index_token']), axis=1)

In [ ]:
tcof_tokens['MD']=tcof_tokens['forme'].apply(lambda x : 1 if x=='oui' else 0)

In [ ]:
tcof_tokens = tcof_tokens.drop('tokens', axis=1)

In [ ]:
tcof_tokens

In [ ]:
import mysql.connector as msql
from mysql.connector import Error

In [ ]:
tcof_tour.columns

In [ ]:
try:
    conn = msql.connect(host='localhost',
                        database='Test3', user='root',
                        password='')
    if conn.is_connected():
        cursor = conn.cursor()

        for i, row in tcof_corpus.iterrows():
            sql =  "INSERT INTO Corpus (id_corpus, nom, date, type) VALUES (%s, %s, %s, %s);"
            cursor.execute(sql, (row['index_corpus'], row['corpus'], row['date'], row['type']))
            print("Record inserted into tour")
        conn.commit()
        print("Data insertion completed.")
except Error as e:
    print("Error while connecting to MySQL", e) 

In [ ]:
try:
    conn = msql.connect(host='localhost',
                        database='Test3', user='root',
                        password='')
    if conn.is_connected():
        cursor = conn.cursor()           
        for i, row in tcof_enregistrement.iterrows():
            sql = "INSERT INTO Enregistrement (id_enregistrement, nom, situation, date, responsable, fichier_transcription, fichier_audio, id_corpus) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);"
            cursor.execute(sql, (row['index_enregistrement'], row['sous_corpus'], row['description'], row['date_enregistrement'], row['responsable'], row['path_trs'],row['path_wav'],row['index_corpus']))
            print("Record inserted into tour")
        conn.commit()
        print("Data insertion completed.")
except Error as e:
    print("Error while connecting to MySQL", e) 

In [ ]:
try:
    conn = msql.connect(host='localhost',
                        database='Test3', user='root',
                        password='')
    if conn.is_connected():
        cursor = conn.cursor()                     
        for i, row in tcof_locuteurs.iterrows():
            sql = "INSERT INTO Locuteur (id_locuteur, nom, age, etude, metier, id_enregistrement) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (row['index_locuteur'], row['speaker_name'], row['age_speaker'], row['etude_speaker'], row['profession_speaker'], row['index_enregistrement']))
            print("Record inserted into tour")
        conn.commit()
        print("Data insertion completed.")
except Error as e:
    print("Error while connecting to MySQL", e) 
    

In [ ]:
try:
    conn = msql.connect(host='localhost',
                        database='Test3', user='root',
                        password='')
    if conn.is_connected():
        cursor = conn.cursor()        
        for i, row in tcof_tour.iterrows():
            sql = "INSERT INTO Tour (id_tour, text, debut, fin, id_locuteur, id_enregistrement) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (row['index_tour'], row['text_clean'], row['startTime'], row['endTime'], row['index_locuteur'], row['index_enregistrement']))
            print("Record inserted into tour")
        conn.commit()
        print("Data insertion completed.")
except Error as e:
    print("Error while connecting to MySQL", e) 

In [53]:
try:
    conn = msql.connect(host='localhost',
                        database='Test3', user='root',
                        password='')
    if conn.is_connected():
        cursor = conn.cursor()
        for i, row in tcof_tokens.iterrows():
            sql = "INSERT INTO Tokens (id_token, id_tour, id_locuteur, num_tour, forme,  md ) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (row['index_token'], row['index_tour'], row['index_locuteur'],row['num_tokens'], row['forme'], row['MD']))          
        
        # Commit the changes
        conn.commit()
        print("Data insertion completed.")
except Error as e:
    print("Error while connecting to MySQL", e)

Data insertion completed.


In [52]:
tcof_tokens.columns

Index(['index_token', 'index_tour', 'index_locuteur', 'forme', 'num_tokens',
       'MD'],
      dtype='object')